In [2]:
from src.data import OxfordPetDatasetLoader
from src.config import Config
from src.data import DataPreprocessor, DataAugmentor


config = Config()
config.USE_IMAGENET_NORM = True          # For transfer learning
config.PRESERVE_ASPECT_RATIO = True      # Better image quality
config.ENABLE_QUALITY_ENHANCEMENT = True # Image enhancement
config.NORMALIZATION_METHOD = 'imagenet' # Best for pretrained models

2025-06-17 20:56:44.187123: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 20:56:44.195186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750168604.204676   81239 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750168604.207351   81239 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750168604.214488   81239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:

loader = OxfordPetDatasetLoader(
    data_dir="./data",  # Tùy chọn
    download=True,
    log_level='INFO'
)

INFO:src.data.dataset_loader:Initialized Oxford Pet Dataset Loader
INFO:src.data.dataset_loader:Data directory: ./data


In [4]:

train_ds, val_ds, test_ds = loader.create_train_val_test_splits(
    val_split=0.2,
    seed=42
)

INFO:src.data.dataset_loader:Creating train/validation/test splits...
INFO:src.data.dataset_loader:Checking dataset availability...
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Dataset already available.
INFO:src.data.dataset_loader:Loading raw dataset splits: ['train', 'test']
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
I0000 00:00:1750168606.304311   81239 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6115 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
INFO:absl:Constructing tf.data.Dataset oxford_iiit_pet for split ['train', 'test'], from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Train set: 3,680 examples
INFO:s

In [5]:
cfg = Config()

In [6]:
train_ds.element_spec

{'file_name': TensorSpec(shape=(), dtype=tf.string, name=None),
 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
 'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None),
 'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'segmentation_mask': TensorSpec(shape=(None, None, 1), dtype=tf.uint8, name=None),
 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}

In [7]:
import tensorflow as tf

preprocessor = DataPreprocessor(config=cfg)
train_ds = train_ds.map(preprocessor.preprocess_sample, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocessor.preprocess_sample, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocessor.preprocess_sample, num_parallel_calls=tf.data.AUTOTUNE)

augmentor = DataAugmentor(config=cfg, target_height=cfg.IMG_SIZE[0], target_width=cfg.IMG_SIZE[1])
train_ds = train_ds.map(augmentor, num_parallel_calls=tf.data.AUTOTUNE)

   

In [8]:
def format_detection(sample):
    image = sample['image']
    target = {
        'bbox': sample['head_bbox'],
        'label': sample['label'],
        'species': sample['species'],
            }
    return image, target
    
output_signature = (
            tf.TensorSpec(shape=(cfg.IMG_SIZE[0], cfg.IMG_SIZE[1], 3), dtype=tf.float32),
            {
                'bbox': tf.TensorSpec(shape=(4,), dtype=tf.float32),
                'label': tf.TensorSpec(shape=(), dtype=tf.int64),
                'species': tf.TensorSpec(shape=(), dtype=tf.int64),
            }
        )
train_ds = train_ds.map(format_detection, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(format_detection, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(format_detection, num_parallel_calls=tf.data.AUTOTUNE)

In [26]:
train_ds.element_spec

(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None),
 {'bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
  'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
  'species': TensorSpec(shape=(), dtype=tf.int64, name=None)})

In [14]:
# Get the sample
sample = next(iter(train_ds))
sample


(<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
 array([[[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],
 
        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],
 
        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],
 
        ...,
 
        [[0.        , 0.        